In [1]:
# Imports
from src.data_retrieval import init_osm
from src.preprocessing import load_roads, load_railways, load_water_polygons, get_local_crs, reproject_all, build_water_edges
from src.plotting import plot_base_map, plot_water_map
from src.prepare_pois import prepare_pois, load_pois_with_green

In [2]:
# CITY_NAME = "Copenhagen"
CITY_NAME = "Gdansk"

# Initialize OSM
osm, boundary = init_osm(CITY_NAME)

In [3]:
# pois = load_pois(osm)
pois = load_pois_with_green(osm)

Loaded 33140 regular POIs
Loaded 251 green-space features
Total combined POIs: 33391


In [4]:
roads = load_roads(osm)
railways = load_railways(osm)
water_polygons = load_water_polygons(osm)

/home/zuziaderylo/anaconda3/envs/masters/lib/python3.11/site-packages/pyrosm/networks.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  edges, nodes = prepare_geodataframe(


In [5]:
# Compute city-local UTM projection
local_epsg = get_local_crs(boundary)
print(f"Using local CRS: EPSG:{local_epsg}")

# Reproject all at once
layers = {
    "boundary": boundary,
    "roads": roads,
    "railways": railways,
    "water_polygons": water_polygons,
    "pois": pois
}
layers = reproject_all(layers, local_epsg)

# Unpack
boundary = layers["boundary"]
roads = layers["roads"]
railways = layers["railways"]
water_polygons = layers["water_polygons"]
pois = layers["pois"]

Using local CRS: EPSG:32634


In [6]:
print(boundary.crs)
print(roads.crs)
print(railways.crs)
print(water_polygons.crs)
print(pois.crs)

EPSG:32634
EPSG:32634
EPSG:32634
EPSG:32634
EPSG:32634


In [7]:
pois = prepare_pois(pois=pois)
pois["category"].value_counts()

Number of POIs after cleaning: 33368
Assigned categories for 19019 POIs (57.0% coverage)
Number of POIs after categorization: 33368


category
infrastructure_transport    7945
retail                      4303
other_daily_utilities       1967
culture_leisure             1949
food                        1243
healthcare                   544
public_services              416
education                    405
green_spaces                 247
Name: count, dtype: int64

In [8]:
# Build water edges after reprojection
water_edges = build_water_edges(layers["water_polygons"])

print(water_edges.crs)

EPSG:32634


In [9]:
# # USE THIS ONE!!!!
# plot_base_map(boundary, roads, railways, water_polygons, water_edges, city_name=CITY_NAME, save_path="report/figures")

In [10]:
# # USE THIS ONE!!!
# plot_water_map(water_polygons, water_edges, boundary = None, city_name=CITY_NAME, save_path="report/figures")